---

# Code pour les flags 11 à 15



---

In [8]:
import subprocess

#on donne en argument à sign le message à signer et le chemin de la clé privée à utiliser pour signer.
def sign(plain_text,chemin_priv_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'dgst', '-sha256', '-sign', chemin_priv_key,'-hex']

    # fichier=open('sign_elec.txt','r')
    message=plain_text
    # si plaintext est de stype str, on est obligé de l'encoder en bytes pour
    # pouvoir l'envoyer dans le pipeline vers openssl
    if isinstance(message, str):
        message=message.encode('utf-8')

    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=message, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        print (error_message)
    

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout



# FLAG 11 
rsa.keygen
(Atrium Lecteur NFC)

Pour rentrer dans l'atrium il est nécessaire d'avoir upload dans sa "card data" une clé publique avec la chaine de caractère "ea+++ATRIUM+++ed" à la fin de celle-ci (depuis un terminal de service). Une fois cela fait, on va à l'atrium et on tappe "ouvrir serrure carte" et on signe avec la clé privée associée à la clé publique (spéciale atrium) qu'on vient de générer

In [9]:
# Importation des modules nécessaires
from Crypto.PublicKey import RSA  # Module pour la génération de clés RSA
import base64  # Module pour le décodage de chaînes base64
import os


def generation_paire_cles_atrium():
    entree = "ea+++ATRIUM+++ed"

    valeur = int.from_bytes(base64.b64decode(entree), byteorder='big')

    element = RSA.generate(2048,randfunc=None, e=valeur)

    sortie_pub=open("keys/pkey_atrium.pem",'wb')
    sortie_pub.write(element.publickey().export_key('PEM'))
    sortie_pub.close()

    sortie_priv=open("keys/priv_atriumkey.pem",'wb')
    sortie_priv.write(element.export_key('PEM'))
    sortie_priv.close()


generation_paire_cles_atrium()
print(sign("gloom obese welds ouzel dunes",'keys/priv_atriumkey.pem'))




b'SHA2-256(stdin)= 862f30bfcb2b56cad6722d08d073d23081bc08fd74015ce8ae1a9e65f4db1c0a2f8c3e498c583aac426b250e594f4758119432ffabd9e372d31458e67d039e1dd46c807a4879d454d0debdca7cd2b824df8cc2142f28b6b8d73e1bbc6b0771088f7e68fcc15a30cc8dab45ef0e6241a0a519b895ca977887724a1efa855c2a1e8ff416a6441d6afee5531d28375349426bbf60cf2160da15db5d5ff8a780c8c149edfabe04582d9308aa26f77e6b2655404124b19a8112b7555b977e40f29993f0a560c34b167368104161725860cd822c811f9b14ac5da83c6348ee4f83f84fc8c5d6910d9470229aad068137dd6ca022b467564a9f10b4b3c334a6d01b5cf8\n'


# FLAG 12 : 
rsa.reduction RC-07 (atrium)


On cherche à entrer en RC-07 mais il y'a encore une serrure. On va se faire passer pour jamieyoung en faisant une signature à sa place lorsqu'on tappe "ouvrir serrure carte" devant la porte RC-07. Pour cela on commene par récupérer sa clé publique et celle de son collègue "phillip"  au terminal de service de l'atrium. On colle, celles-ci dans pkey_keyphillip.pem et pkey_jamieyoung.pem. Ensuite, on se rend compte que Phillipe a laissé trainé le "d" de sa clé privée. Grâce au d on peut obtenir la factorisation de N. Comme on sait que Jamie et Phillipe partage le meme N, on a la factorisation du N de Jamie aussi. Il est donc simple de retrouve le d associé à Jamie et ainsi obtenir sa clé privée complète !

Une fois la clé privée de Jamie obtenue, on upload la clé publique de Jamie sur notre card data et on signe avec sa clé privée le challenge qu'on nous donne.

In [10]:
from Crypto.PublicKey import RSA
from Crypto.Util.number import inverse
import random
import math
import os

# d de Phillipe qui l'a laissé trainé
d = int("46a68642bd211bd0044ee508585d9ce4d6675852e4f52a3bc10e8d364fa835ee8492cc2e6f8981d70cebe756235ee40e5cfab3a7c010c0af816aa6b0f1a883d88a99a7f0a8a2cd785514bf8389a6fed14a18554b1d7b4d6fc0c44b9fcf893f4e484630440224ccf4e2b66e0d810b5dcf33edb5580a8fbf76a2a6d21207944ec2d37d9e35a0c4ab728801f0d8a537f85327d361ca51dae16b51b8c3c2591505e3747259f1db5f8a9a32183cd4d13c9a7b9de3158550cb68b1b0dbea61f3e862d8cb29d05956bb95c7aa4808b195335544c6520d91b2cf88741f3cc7c418e3e4d357e3afd66a6b2024afa53c98e73f0360265e14050a46e35b7812ecd8d140df01",16)

# Chargement d'une clé publique RSA
with open("keys/pkey_keyphillip.pem", 'rb') as f:
    contenu_pub = f.read()
cle_publique = RSA.import_key(contenu_pub)

modulo = cle_publique.n
exp = cle_publique.e

# print("modulo =", modulo)
# print("exposant public =", exp)
# print("longueur modulo =", modulo.bit_length())

def retrouver_facteurs(n, exp, val_sec):
    compteur = 0
    potentiel = exp * val_sec - 1

    # print("valeur potentielle =", potentiel)

    while potentiel % 2 == 0:
        potentiel //= 2
        compteur += 1

    # print("puissance de 2 extraite :", compteur)
    # print("reste =", potentiel)

    essais = 0
    limite = 100

    while essais < limite:
        base = random.randint(2, n - 2)
        # print(f"Essai {essais}: base aléatoire =", base)

        candidate = pow(base, potentiel, n)
        # print("Première puissance =", candidate)

        if candidate not in (1, n - 1):
            etape = 0
            trouve = False
            while etape < compteur and not trouve:
                precedent = candidate
                candidate = pow(candidate, 2, n)
                # print(f"  Étape {etape}: candidate = {candidate}")

                condition_1 = candidate == 1
                condition_2 = candidate == n - 1
                if condition_1:
                    facteur = math.gcd(precedent - 1, n)
                    # print("  → Facteur trouvé:", facteur)
                    if 1 < facteur < n:
                        return (facteur, n // facteur)
                    trouve = True
                elif condition_2:
                    trouve = True
                etape += 1

        essais += 1

    # print("Échec après 100 essais")
    return (0, 0)

# À définir ou récupérer correctement
valeur_secrete = d  # ← ligne à vérifier, hypothèse : d = valeur secrète exploitée ici

fact1, fact2 = retrouver_facteurs(modulo, exp, valeur_secrete)

# print("Facteurs retrouvés :", fact1, fact2)

# Seconde clé publique
with open("keys/pkey_jamieyoung.pem", 'rb') as f:
    contenu_pub2 = f.read()
cle_publique_2 = RSA.import_key(contenu_pub2)

if cle_publique_2.has_private() is False:
    exposant_pub_2 = cle_publique_2.e
    # print("Exposant public Jamie :", exposant_pub_2)

    ordre = (fact1 - 1) * (fact2 - 1)
    # print("Ordre (phi(n)) :", ordre)

    exposant_priv_2 = inverse(exposant_pub_2, ordre)
    # print("Exposant privé reconstitué :", exposant_priv_2)

    cle_complete = RSA.construct((modulo, exposant_pub_2, exposant_priv_2))

    os.makedirs("keys", exist_ok=True)
    with open("keys/privatekey_jamieyoung.pem", 'wb') as sortie:
        sortie.write(cle_complete.export_key('PEM'))
    # print("Clé privée sauvegardée dans keys/privatekey_jamieyoung.pem")


print("signature de Jamie : ", sign("sutra noisy steed sills dizzy", "keys/privatekey_jamieyoung.pem"))


signature de Jamie :  b'SHA2-256(stdin)= 96fac4199ec9f7dfe3a4d00a9ff0cad2f9cb1e693c28342a1dbf86aac4e441c25a56b77a640f367cc0367415f9c31f85eda990c3401254c9f0d107067b4f9c5d8897c4e43f5037a61f2f510aa58823880a763c61090f71199faa00a338ec8ed809e906bcca9d5cd16a8a46e79a1f0f2b7dc75cca5c27bfe80212f6e07b00ccbff830ad4f99f8ffe444e63f680c1d4d3bc7e82aa7d87d64dfb084179f1cd4c9c1079770bfe904d9e5118c23312caa5c301560ba1e5ec58c01c00d5d75201ad00621e8a81ff4f06366a851d98ee7dff9262c13d0cb3f38fbb149ca7eb4bcb4da345c575049da27b7425df1a5fea9f731ec1eb27ff23e2a83154baf7e2afe39157d\n'


# FLAG 13 : RC 19 
secure.vault (LABO Nord ouest)

On se rend en rc-19 au labo. On ouvre le terminal et on copie le chiffré qui apparait dans "message.txt"


La "seed" utilisée dans l’enclave sécurisée fait que 16 bits seulement sont utilisés, ce qui fait seulement 65536 combinaisons possibles. Dans le code, on génère toutes ces combinaisons possibles avec deux octets (bytes([i, j])), et pour chaque graine on reproduit la fonction de dérivation (la key_expansion) pour retrouver la clé AES (128 bits) et l’IV (aussi 128 bits).

Ensuite, on compare l’IV qu’on a généré avec celui qu’on cherche (valeur_iv). Quand ça matche, c’est que la clé est bonne aussi. Du coup on peut directement déchiffrer le message avec AES en mode CBC. Et comme il n’y a que 65536 essais à faire, on peut brute-force.

Enfin, on colle le "Nouveau mot de passe pour la salle d'expériences" dans le digicode du sas du labo.



In [11]:
from hashlib import sha256
from Crypto.Cipher import AES
import base64

# Valeur IV 
valeur_iv = "76edc61bda2704dfc9fc68822b1d0258"

# Création de toutes les paires possibles d'octets (2 octets = 16 bits)
def combinaisons_possibles():
    return [bytes([i, j]) for i in range(256) for j in range(256)]

def expansion_cle(entree: bytes):
    resultat = b''
    courant = entree
    for k in range(8):
        courant = sha256(courant).digest()
        resultat += courant[:4]
    return resultat

# Lecture et décodage du fichier contenant le texte chiffré
with open("message.txt", "r") as fichier_texte:
    contenu = fichier_texte.read()
    bloc_chiffre = base64.b64decode(contenu)

# print("Contenu du fichier déchiffré (Base64):", contenu)
# print("Bloc chiffré (Base64 décodé):", bloc_chiffre.hex())

# Initialisation d’un compteur de résultats
nb_candidats_valides = 0

# Analyse de toutes les combinaisons
for tentative in combinaisons_possibles():
    # print("Tentative actuelle:", tentative.hex())

    donnees = expansion_cle(tentative)
    cle_aes = donnees[:16]
    iv_test = donnees[16:]

    # print("Clé AES générée:", cle_aes.hex())
    # print("IV testé:", iv_test.hex())

    if iv_test.hex() == valeur_iv:
        aes = AES.new(cle_aes, AES.MODE_CBC, iv_test)
        bloc_clair = aes.decrypt(bloc_chiffre)

        # Suppression du padding (basé sur la dernière valeur)
        padding = bloc_clair[-1]
        bloc_final = bloc_clair[:-padding]

        # print("Bloc clair avant suppression du padding:", bloc_clair.hex())
        # print("Padding détecté:", padding)

        print("Message déchiffré", bloc_final.decode(errors='replace'))

        nb_candidats_valides += 1

# print("Nombre de candidats valides:", nb_candidats_valides)


Message déchiffré Cahier de manipulation

09:00 Démarrage de l'accélérateur

09:27 Puissance nominale

09:45 Première tentative de broadcast à cette puissance
      Envoi de "Hello world!"

09:57 Réception depuis les coordonnées db75ed3b34c012ad17974772a76a56a1 
     de la chaine "Qui est là ? Il y a quelqu'un ?".

10:23 Echange de plusieurs autres messages.  Apparemment notre interlocuteur 
      est un être humain situé aux coordonnées db75ed3b34c012ad17974772a76a56a1.

10:45 Légère surtension dans la bobine #3.  Arrêt et inspection des dégats.

14:12 Redémarrage de l'accélérateur

14:58 Puissance maximale.  Tous les systèmes OK.

15:03 Démarrage de la tentative de téléportation d'une brique de légo 
      vers les coordonnées db75ed3b34c012ad17974772a76a56a1.

15:07 Explosion majeure dans la salle d'expérience.  stanley42 évacué par 
      le SAMU (brulures superficielles)

15:09 La brique de légo a disparu (vaporisée par l'explosion ?)

16:20 Réunion d'urgence du conseil de labo.  

# FLAG 14 : RC-13 
hybrid (labo)

Dans la rc-13, sur l'ordinateur de vienna5 on peut broadcast des messages. Mais, le secrétaire veut chiffrer le message avec la technique de chiffrement hybride. Le secrétaire nous donne sa clé publique et il faut qu'on l'utilise pour chiffrer la clé secrète (à usage unique) utilisée pour chiffrer le message en clair lui-même. 
On crée un fichier json avec le chiffré de la clé à usage unique et le chiffré du message.
Une fois connecté à l'ordi, on brocast tout le json.

In [12]:
import secrets
import subprocess
import binascii
import json


def chiffrer_texte(texte, mot_de_passe, algorithme='aes-128-cbc'):
    """
    Chiffre du texte en clair avec OpenSSL en utilisant un mot de passe et un algorithme symétrique.
    """
    commande = [
        'openssl', 'enc', f'-{algorithme}', '-base64',
        '-pass', f'pass:{mot_de_passe}', '-pbkdf2'
    ]

    if isinstance(texte, str):
        texte = texte.encode('utf-8')

    resultat = subprocess.run(commande, input=texte, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if resultat.stderr:
        print(resultat.stderr.decode())

    return resultat.stdout.decode()


def chiffrer_cle_session(cle_publique_path, cle_session):
    """
    Chiffre une clé de session en utilisant une clé publique RSA avec OpenSSL.
    """
    commande = [
        'openssl', 'pkeyutl', '-encrypt',
        '-pubin', '-inkey', cle_publique_path
    ]

    if isinstance(cle_session, str):
        cle_session = cle_session.encode('utf-8')

    resultat = subprocess.run(commande, input=cle_session, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if resultat.stderr:
        print(resultat.stderr.decode())

    cle_chiffree_hex = binascii.hexlify(resultat.stdout).decode()
    return cle_chiffree_hex


# Étape 1 : Générer une clé de session aléatoire (128 bits)
cle_session = secrets.token_hex(16)

# Étape 2 : Enregistrer la clé de session dans un fichier
with open("cle_session.txt", "w") as fichier_cle:
    fichier_cle.write(cle_session)

# Étape 3 : Texte à chiffrer
texte_original = "Je m'appelle Paul"

# Chiffrer le texte avec la clé de session
texte_chiffre = chiffrer_texte(texte_original, cle_session)

# Clé publique pour chiffrer la clé de session
chemin_cle_publique = "keys/pkey_secretaire.pem"

# Chiffrer la clé de session avec la clé publique
cle_session_chiffree = chiffrer_cle_session(chemin_cle_publique, cle_session)

# Regrouper les données dans un dictionnaire
donnees_finales = {
    "session-key": cle_session_chiffree,
    "ciphertext": texte_chiffre
}

# Affichage du résultat au format JSON
print("Résultat :\n", json.dumps(donnees_finales, indent=2))


Résultat :
 {
  "session-key": "94cd98c62b2f7d4021a64319ddfcbb22a8dbc21edef56a9b5c1701752737613a77f8319690e9a3afbc6a1deb3edf8c168952c32498b4ba308389c5925bcddaeeee9c5ac276ab32c3ebd8460d3c62d6e6123e06981af734297a9c811468f4e75692152c0085410967e8496dddb827648c7eceb720c9ba40142a7a505a7fdef499276878ab2c645c61d823dd1ad4cbb379e95ba3309476ac37da6aeb4974c79e484782c98529c55dc2ac006549dd24b3272745c2ad547a36f2175c0c9c38d5212f73ee9cd6e697e1f7069e3f34dcd712fc7d9f75c101087c64672fbeb425588d687777bb640359b97a3f33d131aa4d644350c6052813bd2591b290b40cec6b5b61",
  "ciphertext": "U2FsdGVkX1+8hNYxIE/BA1MBEmfTQQMt5Gm3mOnjqYWvnr4LsY+79IMUsa/olTNO\n"
}


Signature de la clé publique d'Ilona et d'Halex

In [13]:
#Ilona
print(sign("""-----BEGIN PUBLIC KEY-----
MIIBKzANBgkqhkiG9w0BAQEFAAOCARgAMIIBEwKCAQEAsEI0reZShXNWGsjutSxz
BlD1sNe0DXwieOUBQgIib1ugrvoh+tJsbJBzptXC7uG5O8EubDltnJ22FB4IrOZQ
hQtzG0r7mTVe9grvO1peSHTOSOF7agb1oza5Z0BwC8VdlTFaCEeizOczQdKS/0dk
+KaaWbwqQJUTvM/ixQRzs9V1we+M5r9IuVsHpK7dRBdzuitNE/yitbq1GyertRzK
ujWnVB6RAP1sWIRdk0ZdYy3cCzaufHFmKw1mjH2vibWNzUikFObtJVD+bUAuWBQk
qBFpUabpHCz/N8ZcPqM1ySVBux8qF55LvW51Pl2NQIaOkohJ9VfawNy72MsFbWQH
ewIMea+++ATRIUM+++ed
-----END PUBLIC KEY-----""","keys/priv_key.pem"))

#Halex
print(sign("""-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAzGUlIVCjmbijeMvsG9x0
PK5t4mefqrKyxbnIZZbttrW/j2WxGsxrY/9oeLr+9+UHqOsuFdccqscmsN8J8Yx4
HDoId8ivbrK1wa09KdGZoe+r4Z/5R418XO6DYJ71tKri0lPhfST6VYxL7rcwsgqY
8q+C2HBvxFef3P2zJXmiyttUC49KWHEoczvPGmxMid8Q/GlwP0awxqqFTQE+EfGE
zAdkSjoUrDBD0tFicXN78xNSBB9MhehsXKx0I0hdwmScRkH9/IPMY7nwKTLuwzG9
nnHUkJgxpytcwLWsOnn22TsW0UE6e/pI0ilr9jiAlhH7oSlj4iuuc5QXZnUPS668
NQIDAQAB
-----END PUBLIC KEY-----""","keys/priv_key.pem"))

b'SHA2-256(stdin)= 767a727aa221cfb2376105cedfaa143e5e4b7c77777ec141d05252ffda7c1e1e696d2eb24ecf73f2e5c67e669504ec11d8cc4a12c9ced24d9cb9055c0f9d8aa579a55ceebf539d5677a3d2536482d11a7abf1b4d64df7f7694039db61e64c1bbe0db2a875d00a2c5e492bfa9e21a4dd0d90345586ef3343bdd65e2d6d349784013eb98a08a11c8cdd4ed8efa0be7a97c512de849d7a4fd0c9f88f53b6ff17c9d09ec8d3089177ab8352259dad078199409b04f2a4015107cbfb048d1c7d53d29353eec8d821d3cb3c5518c3c515fc97ffd87de6ac7ebe4d179e301331bdadf3c7252a05c1e95650817afa1bcd698ebddfe2f5a8d34a5a2415776105121d01c58\n'
b'SHA2-256(stdin)= 1087c39d116b55d6cbf63c9d54ebd3a9cf28a247720876f9d70b715113069e718e0dd657243ee6d6a7df4b6906d1a179a4082d1edf2e7e39529e25ad2c92a89411e3cfe8fb9b7498ef86b14bb16ffde7dd5d1f5b37b3c4cc82acf70c16031226d07c144fa25708787cf1b9b3ebabf0ab63db2d9dd1c0929120c665bd0daef046cd14e87eec7d75c0139802a6af2e37c345407e122c37d7f867cc4b1914eacaf2a13645263a2a55b2c162ee2f1497449cefda7b93a8ab7c97162d39884944c20c9dc76f756a9d1443478ba183bdb8fe4b69a881cc1a5df21061fb55e3bc4eee

# Flag 15 
web.of.trust

Il faut demander la signature par 2 utilisateurs de ma clé publique (signature par Ilona et Cassandra) puis l'upload sur un terminal de service à l'option Public Key Infrastructure : Upload une signature. Enfin, pour valider le flag on broadcast depuis rc-13 le meme json que pour que le flag 12.